# Structured Output with Pydantic V2

Simple example of getting structured data from agents using Pydantic V2.

## Key Concept
Instead of free-form text → Get validated, structured data objects

## Prerequisites

```bash
pip install --pre pydantic langchain langchain-ollama
ollama pull qwen3
```

In [2]:
from pydantic import BaseModel, Field
from langchain_ollama import ChatOllama
from langchain.agents import create_agent

## Define Data Structure

In [3]:
class ContactInfo(BaseModel):
    name: str
    email: str
    phone: str


class ListofContactInfo(BaseModel):
    contacts: list[ContactInfo] = Field(..., description="List of contact information")

## Test Simple Structured Output

In [6]:
model = ChatOllama(model="qwen3")

agent = create_agent(
    model,
    tools=[], # if case if you don't want to use any tools, just pass an empty list
    response_format=ContactInfo
)

result = agent.invoke({
    "messages":"Extract contact info from: John Doe, john@example.com, (555) 123-4567, Jon Do, john@example.com1, (555) 123"
})

result["structured_response"]

ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')

## Test Complex Structured Output

In [7]:
model = ChatOllama(model="qwen3")

agent = create_agent(
    model,
    tools=[], # if case if you don't want to use any tools, just pass an empty list
    response_format=ListofContactInfo
)

result = agent.invoke({
    "messages":"Extract contact info from: John Doe, john@example.com, (555) 123-4567, Jon Do, john@example.com1, (555) 123"
})

result["structured_response"]

ListofContactInfo(contacts=[ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567'), ContactInfo(name='Jon Do', email='john@example.com1', phone='(555) 123')])